In [2]:
import sys; sys.path.append('..')

import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

from common.polymarket import load_market_data

In [3]:
data = load_market_data(filename="trump-wins-ends-ukraine-war-in-90-days.json")

event_info = data['event']
markets = data['markets']
price_data = data['price_data']

start_date = event_info['start_date']
end_date = event_info['end_date']

print(f"Event: {event_info['title']}")
print(f"Period: {start_date.date()} to {end_date.date()}")
print(f"Markets: {len(markets)}")
print(f"Duration: {(end_date - start_date).days} days")

Event: Trump ends Ukraine war in first 90 days?
Period: 2024-10-22 to 2025-04-20
Markets: 1
Duration: 179 days


In [ ]:
keywords = [
    "ukraine", "war", "russian", "europe", "putin", "zelensky", "ukrainian", "russian",
    "trump", "ceasefire", "peace", "negotiation",
    "deal",
]

major_domains = [
    "nytimes.com",
    "washingtonpost.com",
    "cnn.com",
    "foxnews.com",
    "abcnews.go.com",
    "nbcnews.com",
    "cbsnews.com",
#     "usatoday.com",
    "bloomberg.com",
    "reuters.com",
#     "apnews.com",
    "bbc.com",
    "theguardian.com",
#     "ft.com",
]

all_articles = []

current = start_date
week_delta = timedelta(days=7)

chunk_end = min(current + week_delta, end_date)

start_str = current.strftime('%Y%m%d%H%M%S')
end_str = chunk_end.strftime('%Y%m%d%H%M%S')

for domain in major_domains:
    query = f"sourcelang:English AND domain:{domain} AND ({' OR '.join(keywords)})"

    url = "https://api.gdeltproject.org/api/v2/doc/doc"
    params = {
        'query': query,
        'mode': 'ArtList',
        'maxrecords': 2,
        'format': 'json',
        'startdatetime': start_str,
        'enddatetime': end_str,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200 and response.text.strip():
        articles = response.json().get('articles', [])
        for article in articles:
            article['source_domain'] = domain
            print(article["title"])
        all_articles.extend(articles)
        print(f"{domain} | {current.date()} to {chunk_end.date()}: {len(articles)} articles\n\n")